# Tool use (api calls) w/ Anthropic Claude 3.x versions on Amazon Bedrock

In this notebook, we demonstrate how to build tools that make calls to external apis to get information.

Ref: <https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/calculator_tool.ipynb>

Install packages using uv, an extremely fast python package installer\
Read more about uv here https://astral.sh/blog/uv

In [ ]:
%%bash
pip install uv && uv pip install -U boto3 rich

In [2]:
!python --version

Python 3.10.14


In [3]:
# restart kernel
from IPython.core.display import HTML

HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
%load_ext rich

In [5]:
import json
import types
from datetime import date

import boto3
from tools_claudev3 import *
from rich import print

## List Anthropic Model IDs in Bedrock

In [6]:
session = boto3.Session()
region = session.region_name
bedrock = session.client(service_name='bedrock', region_name=region)
bedrock_runtime = session.client(service_name='bedrock-runtime', region_name=region)

# List Anthropic models in Bedrock
models = bedrock.list_foundation_models(byProvider="Anthropic", byOutputModality="TEXT")[
    "modelSummaries"
]

# Print only Claude 3 models
model_ids = [
    model["modelId"]
    for model in models
    if "claude-3" in model["modelId"] and "v1:0:" not in model["modelId"]
]

print("[b blue]Claude 3 ModelIDs")
print(model_ids)

print("===" * 10)
# change string in if loop for sonnet or opus
claude_haiku = [m for m in model_ids if "haiku" in m][0]
print(f"Haiku Model ID: [b red]{claude_haiku}")
print("===" * 10)

Claude 3 ModelIDs

[
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0',
    'anthropic.claude-3-opus-20240229-v1:0'
]

==============================

Haiku Model ID: anthropic.claude-3-haiku-20240307-v1:0

==============================

In [7]:
# Invoking using Anthropic Client + Session Credentials

# creds = boto_session.get_credentials()

# llm_client = AnthropicBedrock(
#     aws_access_key=creds.access_key,
#     aws_secret_key=creds.secret_key,
#     aws_session_token=creds.token,
#     aws_region=region,
# )

# message = llm_client.messages.create(
#     model=claude_haiku,
#     max_tokens=256,
#     messages=[{"role": "user", "content": "Hello, world"}],
# )
# print(type(message))
# print(message.content)

## Define tools

1. We'll define a simple `calculator` tool that can perform basic arithmetic operations. The tool will take a mathematical expression as input and return the result.
2. Next, We define two additional functions to help get weather information
   1. `get_lat_long` to get latitude and longitude for a given place
   2. `get_weather` gets the current weather for a given latitude and longitude

<div class="alert alert-info">
<strong>Note:</strong> Refer to <a href="./tools_claudev3.py">tools_claudev3.py</a> file for all defined tools.
</div>

### Example 1. Calculator Tool

In this example, we define a calculate function that takes a mathematical expression as input, removes any non-digit or non-operator characters using a regular expression, and then evaluates the expression using the built-in `eval()` function.\
If the evaluation is successful, the result is returned as a string. If an error occurs during evaluation, an error message is returned.

We then define the calculator tool with an input schema that expects a single expression property of type string.

**Note:** We call `eval` on the outputted expression.\
This is bad practice and should not be used generally but we are doing it for the purpose of demonstration.

In [8]:
# %cat tools_claudev3.py

In [9]:
# Let's examine the tools list we pass to claude v3 as model parameters
print(tools)

[
    {
        'name': 'calculator',
        'description': 'A simple calculator that performs basic arithmetic operations.',
        'input_schema': {
            'type': 'object',
            'properties': {
                'expression': {
                    'type': 'string',
                    'description': "The mathematical expression to evaluate (e.g., '2 + 3 * 4')."
                }
            },
            'required': ['expression']
        }
    },
    {
        'name': 'get_lat_long',
        'description': 'Tool to retrieve latitude and longitude for a given place.',
        'input_schema': {
            'type': 'object',
            'properties': {
                'place': {
                    'type': 'string',
                    'description': 'The place name to geocode and get coordinates for. (e.g., San Francisco)'
                }
            },
            'required': ['place']
        }
    },
    {
        'name': 'get_weather',
        'description': 'Returns weather data for a given latitude and longitude.',
        'input_schema': {
            'type': 'object',
            'properties': {
                'latitude': {'type': 'string', 'description': 'The latitude coordinate as a string'},
                'longitude': {'type': 'string', 'description': 'The longitude coordinate as a string'}
            },
            'required': ['latitude', 'longitude']
        }
    }
]

### Helper functions to interact with Claude

Here we use `bedrock_runtime.invoke_model` to invoke the LLM.

In [10]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "calculator":
        return calculate(tool_input["expression"])
    if tool_name == "get_lat_long":
        return get_lat_long(tool_input["place"])
    if tool_name == "get_weather":
        return get_weather(tool_input["latitude"], tool_input["longitude"])


def chat_with_claude(prompt, MODEL_NAME=claude_haiku, tools=tools, bedrock_runtime=bedrock_runtime):
    print(f"\n{'='*50}\nUser Message: {prompt}\n{'='*50}")

    system_prompt = f"""
        Answer as many questions as you can using your existing knowledge.
        For arthimetic operations, always use the calculator tool.
        For getting latest weather information, always use get_lat_long followed by get_weather tools.
        Today's date is {date.today().strftime("%B %d %Y")}
        If you think a user's question involves something in the future that hasn't happened yet, use the search tool.
    """.strip()

    payload = {
        "max_tokens": 4096,
        "anthropic_version": "bedrock-2023-05-31",
        "system": system_prompt,
        "messages": [{"role": "user", "content": prompt}],
        "tool_choice": {"type": "auto"},
        "tools": tools,
    }

    response = bedrock_runtime.invoke_model(body=json.dumps(payload), modelId=MODEL_NAME)
    # read byte stream and load the response object (dict)
    response_body = json.loads(response.get("body").read())
    # SimpleNamespace to make dict dot accessible
    message = types.SimpleNamespace(**response_body)

    print("\nInitial Response:")
    print(f"Stop Reason: {message.stop_reason}")
    print(f"Content:\n{json.dumps(message.content, indent=2)}")

    while True:
        if message.stop_reason == "tool_use":
            tool_use = next((obj for obj in message.content if obj["type"] == "tool_use"), None)
            tool_name = tool_use["name"]
            tool_input = tool_use["input"]
            tool_use_id = tool_use["id"]

            print(f"\nTool Used: {tool_name}")
            print(f"Tool Input: {tool_input}")

            tool_result = process_tool_call(tool_name, tool_input)

            print(f"Tool Result:\n{json.dumps(tool_result, indent=2)}")

            # append the tool_result as a user response
            messages = [
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": message.content},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use_id,
                            "content": json.dumps(tool_result),
                        }
                    ],
                },
            ]
            # update messages in payload with new messages object
            payload_ns = types.SimpleNamespace(**payload)
            payload_ns.messages = messages
            # convert SimpleNamespace object back to dict
            payload = vars(payload_ns)
            response = bedrock_runtime.invoke_model(body=json.dumps(payload), modelId=MODEL_NAME)
            response = json.loads(response.get("body").read())
            message = types.SimpleNamespace(**response)
        else:
            response = message
            break

    final_response = next(
        (obj["text"] for obj in response.content if obj["type"] == "text"),
        None,
    )

    return final_response

### Test Calculator Tool

In [11]:
calculator_result = chat_with_claude("What is the result of 1,984,135 * 9,343,116?")
print(f"\n{'='*50}\nFinal Response:\n{'='*50}")
print(calculator_result)

# calculator_result = chat_with_claude("Calculate (12851 - 593) * 301 + 76")
# print(f"\n{'='*50}\nFinal Response:\n{'='*50}")
# print(calculator_result)

# calculator_result = chat_with_claude("What is 15910385 divided by 193053?")
# print(f"\n{'='*50}\nFinal Response:\n{'='*50}")
# print(calculator_result)

==================================================
User Message: What is the result of 1,984,135 * 9,343,116?
==================================================

Initial Response:

Stop Reason: tool_use

Content:
[
  {
    "type": "tool_use",
    "id": "toolu_bdrk_01Rcu1xjUcxUNE4hPkprHzZ9",
    "name": "calculator",
    "input": {
      "expression": "1984135 * 9343116"
    }
  }
]

Tool Used: calculator

Tool Input: {'expression': '1984135 * 9343116'}

Tool Result:
"18538003464660"

==================================================
Final Response:
==================================================

Okay, based on the calculator tool, the result of 1,984,135 * 9,343,116 is 18,538,003,464,660.

### Test Weather tool

In [12]:
weather_info = chat_with_claude("What is weather in Los Angeles right now?")
print(f"\n{'='*50}\nFinal Response:\n{'='*50}")
print(weather_info)

==================================================
User Message: What is weather in Los Angeles right now?
==================================================

Initial Response:

Stop Reason: tool_use

Content:
[
  {
    "type": "text",
    "text": "Okay, let's find the weather in Los Angeles right now."
  },
  {
    "type": "tool_use",
    "id": "toolu_bdrk_01HQFyzdDGEAUfKde4ET6A4Q",
    "name": "get_lat_long",
    "input": {
      "place": "Los Angeles"
    }
  }
]

Tool Used: get_lat_long

...

Tool Used: get_weather

Tool Input: {'latitude': '34.007287779437', 'longitude': '-118.257720585538'}

Tool Result:
{
  "latitude": 34.001816,
  "longitude": -118.251465,
  "generationtime_ms": 0.050067901611328125,
  "utc_offset_seconds": 0,
  "timezone": "GMT",
  "timezone_abbreviation": "GMT",
  "elevation": 60.0,
  "current_weather_units": {
    "time": "iso8601",
    "interval": "seconds",
    "temperature": "\u00b0C",
    "windspeed": "km/h",
    "winddirection": "\u00b0",
    "is_day": "",
    "weathercode": "wmo code"
  },
  "current_weather": {
    "time": "2024-07-25T18:15",
    "interval": 900,
    "temperature": 30.9,
    "windspeed": 6.6,
    "winddirection": 225,
    "is_day": 1,
    "weathercode": 0
  }
}

==================================================
Final Response:
==================================================

According to the weather data, the current weather in Los Angeles as of July 25 2024 at 6:15pm is:

- Temperature: 30.9°C
- Wind speed: 6.6 km/h
- Wind direction: 225°
- Weather code: 0 (clear sky)

The weather is clear and warm in Los Angeles right now.